#Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocessing Gambar dari Dataset

In [ ]:
TrainingImagePath='/content/drive/MyDrive/Kecerdasan Buatan/Face Images/Final Training Images'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set.class_indices

# Mapping ID dan Nama Wajah Sebagai Output Neurons

In [ ]:
TrainClasses=training_set.class_indices

ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

print("Mapping of Face and its ID",ResultMap)

OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

# Building dan Training Model

In [ ]:
# Membuat CNN deep learning model
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

classifier= Sequential()
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(OutputNeurons, activation='softmax'))
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
StartTime=time.time()

from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1)

classifier.fit(
                    training_set,
                    epochs=60,
                    validation_data=test_set,
                    )

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

# Pengujian Inferensi

In [ ]:
#Via Upload Google Colab

import numpy as np
import keras.utils as image
from google.colab import files
import matplotlib.pyplot as plt

uploaded = files.upload()

for uploads in uploaded.keys():

  ImagePath = uploads
  test_image=image.load_img(ImagePath,target_size=(64, 64))
  test_image=image.img_to_array(test_image)
  test_image=np.expand_dims(test_image,axis=0)
  result=classifier.predict(test_image,verbose=0)

  img = plt.imread(uploads)
  plt.imshow(img)
  plt.show()

  print(uploads)
  print('****'*10)
  print('Prediction is: ',ResultMap[np.argmax(result)])

In [ ]:
#Via Google Colab Content

import numpy as np
import keras.utils as image
import matplotlib.pyplot as plt

ImagePath = '/content/drive/MyDrive/Kecerdasan Buatan/Face Images/Final Training Images/Ryu Min/image_0434_Face_1.jpg'

img = plt.imread(ImagePath)
plt.imshow(img)

test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)

print('****'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

# Prediksi Wajah dari Foto Webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import keras.utils as image
from IPython.display import Image

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  display(Image(filename))
except Exception as err:
  print(str(err))


ImagePath='/content/photo.jpg'

im = Img.open(ImagePath)
im.show()

test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)

print('****'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

# Prediksi Wajah Real-Time via Webcam

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

def js_to_image(js_reply):

  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def bbox_to_bytes(bbox_array):

  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
video_stream()

from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
screen_width = 1280
screen_height = 720

image_width = 64
image_height = 64
model = classifier

label_html = 'Capturing...'
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    frame = img
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = face_cascade.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        roi_rgb = rgb[y:y+h, x:x+w]

        color = (255, 0, 0)
        stroke = 2
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, stroke)

        size = (image_width, image_height)
        resized_image = cv2.resize(roi_rgb, size)
        img = resized_image.reshape(64, 64, 3)
        image_array = np.array(img, "uint8")
        img = img.astype('float32')

        # ImagePath= img;
        # test_image=image.load_img(ImagePath,target_size=(64, 64))
        # test_image=image.img_to_array(test_image)

        test_image=np.expand_dims(img ,axis=0)
        result=classifier.predict(test_image,verbose=0)
        print('Prediction is: ',ResultMap[np.argmax(result)])

        # Display the label
        # font = cv2.FONT_HERSHEY_SIMPLEX
        # name = ResultMap[np.argmax(result)]
        # color = (255, 0, 255)
        # stroke = 2
        # cv2.putText(frame, f'({name})', (x,y-8),
        #     font, 1, color, stroke, cv2.LINE_AA)

        # Show the frame
        # cv2_imshow(frame)

In [ ]:
from PIL import Image
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier(
    '/content/haarcascade_frontalface_default.xml')

screen_width = 1280
screen_height = 720
image_width = 64
image_height = 64


model = classifier

label_html = 'Capturing...'
bbox = ''

stream = video_stream()

while(True):
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])

    frame = img
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = face_cascade.detectMultiScale(
        rgb, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_rgb = rgb[y:y+h, x:x+w]

        color = (255, 0, 0)
        stroke = 2
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, stroke)

        size = (image_width, image_height)
        resized_image = cv2.resize(roi_rgb, size)
        img = resized_image.reshape(64, 64, 3)
        image_array = np.array(img, "uint8")
        img = img.astype('float32')

        result=classifier.predict(test_image,verbose=0)

        font = cv2.FONT_HERSHEY_SIMPLEX
        name = ResultMap[np.argmax(result)]
        color = (255, 0, 255)
        stroke = 2
        cv2.putText(frame, f'({name})', (x,y-8),
            font, 1, color, stroke, cv2.LINE_AA)

        cv2_imshow(frame)